In [117]:
import pandas as pd
import boto3

In [118]:
#Getting access key and id: 
key = pd.read_csv('credentials.csv')
key_id = key['Access key ID'].values[0]
access_key = key['Secret access key'].values[0]

In [119]:
photo = '01.jpg'

In [120]:
client = boto3.client('rekognition',aws_access_key_id=key_id,aws_secret_access_key=access_key,region_name='us-east-2')

In [121]:
# S3 bucket for uploading images into it.
s3 = boto3.client('s3',aws_access_key_id=key_id,aws_secret_access_key=access_key)
response = s3.list_buckets()
buckets = [bucket['Name'] for bucket in response['Buckets']]
print("Bucket List: %s" % buckets)



#filename = '01.jpg'
#bucket_name = 'sonuresob1'
#newname = 'new.jpg'
#s3.upload_file(filename, bucket_name, newname)


Bucket List: ['sonuresob1']


In [112]:
#s3.create_bucket(Bucket='sonuresob2')


   1. Label Detection:  Converting Images to its byte format

In [87]:
with open(photo,'rb') as source_image:
    source_bytes = source_image.read()

    Detecting Labels using bytes of images on local machine.

In [100]:
response = client.detect_labels(Image={'Bytes': source_bytes}, MaxLabels=10,MinConfidence=95)

In [101]:
response

{'Labels': [{'Name': 'Car',
   'Confidence': 99.82917022705078,
   'Instances': [{'BoundingBox': {'Width': 0.21661338210105896,
      'Height': 0.2922539710998535,
      'Left': 0.426487535238266,
      'Top': 0.702470064163208},
     'Confidence': 99.82917022705078},
    {'BoundingBox': {'Width': 0.1579536497592926,
      'Height': 0.1903790831565857,
      'Left': 0.5933496356010437,
      'Top': 0.5612152814865112},
     'Confidence': 98.67333984375},
    {'BoundingBox': {'Width': 0.05757751315832138,
      'Height': 0.086023710668087,
      'Left': 0.8268179893493652,
      'Top': 0.5275430679321289},
     'Confidence': 97.54666137695312},
    {'BoundingBox': {'Width': 0.0489080436527729,
      'Height': 0.08776109665632248,
      'Left': 0.32941025495529175,
      'Top': 0.442045122385025},
     'Confidence': 97.23896789550781},
    {'BoundingBox': {'Width': 0.0354219451546669,
      'Height': 0.051243845373392105,
      'Left': 0.47551241517066956,
      'Top': 0.4181915819644928

    Detecting Using S3 Bucket:

In [116]:
response = client.detect_labels(Image={'S3Object':{'Bucket': 'sonuresob1','Name': photo}},
                                MaxLabels=10,MinConfidence=95)

In [ ]:
response

   2. Face recial Analysis: 

In [125]:
response2 = client.detect_faces(Image={'S3Object':{'Bucket': 'sonuresob1','Name': '11.jpg'}},Attributes=['ALL'])

In [126]:
#use loop in case multiple people in the image:
for key,value in response2.items():
    if key=='FaceDetails':
        for people_att in value:
            print(people_att.keys())
            print('---------------------------')
            print('Age Range: ',people_att['AgeRange'])
            print('Gender: ',people_att['Gender'])
            print('Smile: ',people_att['Smile'])
            print('Emotions: ',people_att['Emotions'])
            print('=============================================\n')

dict_keys(['BoundingBox', 'AgeRange', 'Smile', 'Eyeglasses', 'Sunglasses', 'Gender', 'Beard', 'Mustache', 'EyesOpen', 'MouthOpen', 'Emotions', 'Landmarks', 'Pose', 'Quality', 'Confidence'])
---------------------------
Age Range:  {'Low': 26, 'High': 43}
Gender:  {'Value': 'Male', 'Confidence': 99.9787368774414}
Smile:  {'Value': False, 'Confidence': 99.57845306396484}
Emotions:  [{'Type': 'SAD', 'Confidence': 3.511714220046997}, {'Type': 'SURPRISED', 'Confidence': 1.2579584121704102}, {'Type': 'ANGRY', 'Confidence': 5.19409704208374}, {'Type': 'CALM', 'Confidence': 59.725093841552734}, {'Type': 'HAPPY', 'Confidence': 0.8363337516784668}, {'Type': 'DISGUSTED', 'Confidence': 1.610154151916504}, {'Type': 'CONFUSED', 'Confidence': 27.864646911621094}]



   3. Comparing two images for the match:

In [82]:
response3 = client.compare_faces(SourceImage={'S3Object': {'Bucket': 'sonuresob1','Name': '11.jpg'}}, 
                                TargetImage={'S3Object': {'Bucket': 'sonuresob1','Name': '07.jpg'}})

In [83]:
response3

{'SourceImageFace': {'BoundingBox': {'Width': 0.1779618263244629,
   'Height': 0.3592563271522522,
   'Left': 0.5863519906997681,
   'Top': 0.15845666825771332},
  'Confidence': 99.99998474121094},
 'FaceMatches': [{'Similarity': 96.88172149658203,
   'Face': {'BoundingBox': {'Width': 0.12957529723644257,
     'Height': 0.29724517464637756,
     'Left': 0.4368344843387604,
     'Top': 0.22759073972702026},
    'Confidence': 99.99998474121094,
    'Landmarks': [{'Type': 'eyeLeft',
      'X': 0.4551735520362854,
      'Y': 0.3384191393852234},
     {'Type': 'eyeRight', 'X': 0.5117777585983276, 'Y': 0.3355780243873596},
     {'Type': 'mouthLeft', 'X': 0.4620724022388458, 'Y': 0.4403378665447235},
     {'Type': 'mouthRight', 'X': 0.5084807276725769, 'Y': 0.43834608793258667},
     {'Type': 'nose', 'X': 0.4729882776737213, 'Y': 0.387184739112854}],
    'Pose': {'Roll': -6.285496711730957,
     'Yaw': -24.9992733001709,
     'Pitch': 10.211874008178711},
    'Quality': {'Brightness': 87.6116